In [191]:
# Get top 1500 companies by Market Cap. Using holdings from ITOT. What is ITOT? ITOT is iShares (aka Blackrock) Total US Stock Market exchange traded fund.
# Link to fact sheet here https://www.ishares.com/us/literature/fact-sheet/itot-ishares-core-s-p-total-u-s-stock-market-etf-fund-fact-sheet-en-us.pdf
# Link to Prospectus here: https://www.ishares.com/us/library/stream-document?stream=reg&product=I-SP1500&shareClass=NA&documentId=925902%7E925940%7E926348%7E925661%7E925593&iframeUrlOverride=%2Fus%2Fliterature%2Fprospectus%2Fp-ishares-core-s-and-p-total-us-stock-market-etf-3-31.pdf
# Do I need to pull data for all 3,600 holdings? Just top 1,500 should do. Lets see. 

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import requests
pd.options.display.float_format = '{:,.2f}'.format
import time

In [2]:
time.sleep(3)

In [2]:
base_url = 'https://cloud.iexapis.com/stable'
token = 'pk_8d0f8e5a2a134ad48410c868b4849d70'
sb_token = 'Tpk_8ef08bcd612444eab903cf6d1877b2bb'
sb_url = 'https://sandbox.iexapis.com/stable'


In [3]:
itot_holdings = pd.read_csv('ITOT_holdings_csv.csv', usecols =['Ticker','Name','Sector','Market Value'])

In [9]:
# Use market clean formula, as some market values were expressed as strings
itot_holdings['Market Value'] = itot_holdings['Market Value'].apply(clean_market_value).sort_values(ascending = False)

In [8]:
def clean_market_value(market_value):
    try:
       market_value = float(market_value)
    except:
        market_value = float(market_value.replace(',',''))
    return market_value

In [10]:
# Largest 1500 companies in ITOT
itot_top_1500 = itot_holdings.sort_values('Market Value', ascending = False)[:1500]

In [11]:
# Reset index so '0' is highest market cap, '1499' is lowerst market cap. 
itot_top_1500.reset_index(drop = True, inplace = True)

In [12]:
# Top 1500 holidngs does NOT represent market valuation for companies, rather how much the fund holds. Assuming ITOT strives to reflect
# S&P 1500 as much as possible, assumption is top 1500 holdings will be good approximation for S&P1500, even though ITOT fund has ~3,600 holdings

itot_top_1500

,Ticker,Name,Market Value,Sector
0,MSFT,MICROSOFT CORP,"1,126,359,706.56",Information Technology
1,AAPL,APPLE INC,"1,000,018,562.85",Information Technology
2,AMZN,AMAZON COM INC,"822,291,123.02",Consumer Discretionary
3,FB,FACEBOOK CLASS A INC,"388,864,892.43",Communication
4,GOOGL,ALPHABET INC CLASS A,"334,647,081.58",Communication
...,...,...,...,...
1495,DDD,3D SYSTEMS CORP,"896,904.96",Information Technology
1496,BHE,BENCHMARK ELECTRONICS INC,"895,867.41",Information Technology
1497,XHR,XENIA HOTELS RESORTS REIT INC,"895,864.76",Real Estate
1498,FBP,FIRST BANCORP,"895,265.70",Financials


In [13]:
# How to break into chunks? 
# How to insert chunks into url?
# Solution: Using format. See below
test_url = base_url + '/stock/market/batch?symbols={}&types=chart,quote,company,stats&range=day&exactDate=20200302&chartByDay=true&filter=industry,chart,latestPrice,marketcap&token=' + token
test_url.format('aapl,amzn,goog')

'https://cloud.iexapis.com/stable/stock/market/batch?symbols=aapl,amzn,goog&types=chart,quote,company,stats&range=day&exactDate=20200302&chartByDay=true&filter=industry,chart,latestPrice,marketcap&token=pk_8d0f8e5a2a134ad48410c868b4849d70'

In [14]:
# How to change list, or pd.series, into string?
stock_list = ['aapl','amzn','goog']
string = ','.join(stock_list)

In [15]:
string

'aapl,amzn,goog'

In [16]:
ticker_list = list(itot_top_1500['Ticker'].apply(lambda x: x.lower()))

In [204]:
# How to split up tickers into manageble chunks?
for i in range(0,len(ticker_list),100):
    print(','.join(ticker_list[i: i + 100]))
# See above!! Gets list of tickers, which can be passed to 'symbols' parameter in url

msft,aapl,amzn,fb,googl,goog,jnj,brkb,v,jpm,pg,unh,intc,ma,hd,vz,t,pfe,mrk,dis,xom,bac,pep,csco,nvda,ko,nflx,cvx,cmcsa,wmt,adbe,abt,pypl,bmy,crm,mcd,amgn,cost,tmo,mdt,lly,abbv,acn,pm,orcl,wfc,nee,tsla,ibm,unp,txn,nke,c,avgo,gild,dhr,amt,hon,lin,rtx,lmt,sbux,qcom,mmm,fis,low,cvs,chtr,mo,ba,mdlz,ci,spgi,antm,intu,bdx,ups,pld,cat,cme,d,cci,axp,vrtx,bkng,adp,duk,amd,tjx,agn,syk,now,so,blk,gs,cl,isrg,fisv,zts,eqix
ge,tgt,usb,mu,biib,csx,tfc,bsx,noc,gpn,nem,hum,cb,ice,apd,amat,atvi,ecl,pnc,itw,mmc,nsc,kmb,ms,cop,ilmn,bax,pgr,de,ew,dg,schw,shw,lhx,adi,regn,adsk,mco,aep,lrcx,aon,wm,el,dlr,cnc,sre,exc,gis,emr,etn,bx,dd,rop,rost,all,wba,tmus,xel,ea,sbac,psx,ctsh,xtsla,cof,gd,bk,dxcm,fdx,kmi,syy,hca,orly,ebay,gm,usd,wec,afl,met,msci,stz,psa,klac,dow,lulu,eog,info,vlo,aph,trv,trow,ed,es,peg,yum,tel,mar,iqv,msi,pru,slb
kr,wday,cmi,vrsk,azo,veev,zbh,a,jci,mnst,twtr,pcar,snps,hpq,alxn,stt,csgp,aig,idxx,wmb,mck,avb,xlnx,clx,eqr,fe,cdns,cmg,rok,ppg,hlt,awk,wltw,sq,anss,cern,bll,payx,rmd,mchp,vrsn,splk,

In [205]:
# This should be what each chunk begins with. See above to confirm. It does. 
ticker_list[0:1500:100]

['msft',
 'ge',
 'kr',
 'sgen',
 'qrvo',
 'rcl',
 'zion',
 'ipgp',
 'jcom',
 'bfa',
 'rh',
 'cbt',
 'pgre',
 'sfbs',
 'ibp']

In [206]:
for i in range(0,len(ticker_list),100):
    print(len(ticker_list[i: i + 100]))

100
100
100
100
100
100
100
100
100
100
100
100
100
100
100


In [207]:
# Cleaning returned JSON from IEX API
def clean_json_df(row):
    row['company'] = row['company']['industry']
    row['stats'] = row['stats']['marketcap']
    row['quote'] = row['quote']['latestPrice']
    row['chart'] = row['chart'][0]['close']
    return row
# pd.DataFrame(aapl_amazn_goog_pract).T. Need to transpose dataframe
# pd.DataFrame(aapl_amazn_goog_pract).T.apply(clean_json_df, axis = 1).rename(columns = lambda x: x.title()) Final dataframe call

In [20]:
def clean_json_df_two(row):
    row['company'] = row['company']['industry']
    row['stats'] = row['stats']['marketcap']
    row['quote'] = row['quote']['latestPrice']
    try:
        row['chart'] = row['chart'][0]['close']
    except:
        row['chart'] = 0
    return row

In [208]:
# testing sandbox
requests.get(sb_url + '/stock/brk.b/quote?token=' + sb_token).json()

{'symbol': 'BRK.B',
 'companyName': 'Berkshire Hathaway, Inc.',
 'primaryExchange': 'NxergwoYnk  c ocEtakeSh',
 'calculationPrice': 'close',
 'open': 186.16,
 'openTime': 1601575866573,
 'openSource': 'olfciifa',
 'close': 184.41,
 'closeTime': 1629772101538,
 'closeSource': 'laoiifcf',
 'high': 190.52,
 'highTime': 1622937429414,
 'highSource': 'el ipm tduedr5i1yeca ne',
 'low': 184.17,
 'lowTime': 1629795692286,
 'lowSource': 'ii uy5aree1ptdl cdee nm',
 'latestPrice': 190.63,
 'latestSource': 'Close',
 'latestTime': 'May 1, 2020',
 'latestUpdate': 1609867541501,
 'latestVolume': 7860758,
 'iexRealtimePrice': None,
 'iexRealtimeSize': None,
 'iexLastUpdated': None,
 'delayedPrice': 182.89,
 'delayedPriceTime': 1628637429813,
 'oddLotDelayedPrice': 188.74,
 'oddLotDelayedPriceTime': 1591803520633,
 'extendedPrice': 190.2,
 'extendedChange': 1.74,
 'extendedChangePercent': 0.00975,
 'extendedPriceTime': 1591175104667,
 'previousClose': 188.81,
 'previousVolume': 8473626,
 'change': -4.9

In [24]:
def iex_dataframe(json):
    # Accepts JSON object from IEX API call and return data frame
    df = pd.DataFrame(json).T.apply(clean_json_df, axis = 1).rename(columns = lambda x: x.title())
    return df


In [25]:
def iex_dataframe_two(json):
    df = pd.DataFrame(json).T.apply(clean_json_df_two, axis = 1).rename(columns = lambda x: x.title())
    return df

In [210]:
practice_list = ticker_list[100:200]
practice_url = ','.join(practice_list)

In [211]:
json = requests.get(sb_url + '/stock/market/batch?symbols={}&types=chart,quote,company,stats&range=day&exactDate=20200302&chartByDay=true&filter=industry,chart,latestPrice,marketcap&token='.format(practice_url) + sb_token).json()

In [212]:
iex_dataframe(json)

,Company,Stats,Quote,Chart
GE,rnefDoeesA cspe a&e,56944234687,6.60,11.69
TGT,yrtlasei SoectSp,55796304127,110.42,112.98
USB,naMkrBjaos,54186936905,36.11,50.01
MU,usoorcdemSitnc,50230492697,46.39,55.20
BIIB,cBoooingeylth,48152829300,302.73,342.81
...,...,...,...,...
MAR,rs/tRutene/oslisseClHeirso,27909139832,88.86,124.24
IQV,a yourrnhHthe tedtetsIciSle sv,26963501779,137.58,153.51
MSI,neA& fsa Desceperoe,24617481059,141.33,173.64
PRU,ooemnlacnreisCtFaangil,24261453601,59.84,81.30


In [235]:
itot_top_1500[itot_top_1500['Ticker'] == 'BRK.B']

,Ticker,Name,Market Value,Sector
7,BRK.B,BERKSHIRE HATHAWAY INC CLASS B,"308,626,300.90",Financials


In [17]:
itot_top_1500['Ticker'] = itot_top_1500['Ticker'].str.replace('BRKB','BRK.B',regex = False)

In [18]:
itot_top_1500

,Ticker,Name,Market Value,Sector
0,MSFT,MICROSOFT CORP,"1,126,359,706.56",Information Technology
1,AAPL,APPLE INC,"1,000,018,562.85",Information Technology
2,AMZN,AMAZON COM INC,"822,291,123.02",Consumer Discretionary
3,FB,FACEBOOK CLASS A INC,"388,864,892.43",Communication
4,GOOGL,ALPHABET INC CLASS A,"334,647,081.58",Communication
...,...,...,...,...
1495,DDD,3D SYSTEMS CORP,"896,904.96",Information Technology
1496,BHE,BENCHMARK ELECTRONICS INC,"895,867.41",Information Technology
1497,XHR,XENIA HOTELS RESORTS REIT INC,"895,864.76",Real Estate
1498,FBP,FIRST BANCORP,"895,265.70",Financials


In [22]:
ticker_list = list(itot_top_1500['Ticker'].apply(lambda x: x.lower()))
"""
ticker_list.remove('otis')
ticker_list.remove('carr')
ticker_list.remove('ramp')
ticker_list.remove('ui')
ticker_list.remove('onto')
ticker_list.remove('msge')
"""

"\nticker_list.remove('otis')\nticker_list.remove('carr')\nticker_list.remove('ramp')\nticker_list.remove('ui')\nticker_list.remove('onto')\nticker_list.remove('msge')\n"

In [23]:
ticker_list[0]

'msft'

In [26]:
d = pd.DataFrame()
for i in range(0,len(ticker_list),50):
    url = ','.join(ticker_list[i: i + 50])
    json = requests.get(sb_url + '/stock/market/batch?symbols={}&types=chart,quote,company,stats&range=day&exactDate=20200302&chartByDay=true&filter=industry,chart,latestPrice,marketcap&token='.format(url) + sb_token).json()
    df = iex_dataframe_two(json)
    d = d.append(df)

In [27]:
d

,Company,Stats,Quote,Chart
MSFT,drg oetPaSwcfaake,1386954925170,189.56,178.74
AAPL,uetnomlcit eecoqnETsmnaiipum,1330006197309,302.37,307.50
AMZN,eletRe trntaniI,1166606044268,"2,419.40","2,032.95"
FB,IarSrrenfwe/cvi oeSensettt,587762754656,209.02,200.55
GOOGL,ttweore/inrnaStIvef reSsec,916727078399,"1,395.07","1,409.89"
...,...,...,...,...
DDD,irIasMuleryc dnintah,990577794,8.48,9.69
BHE,rCeomncEtlientsncpo o,734308175,19.54,27.86
XHR,etEnsvl nRastets tsTteIaerum,1045945709,8.70,15.89
FBP,snBoglnakieaR,1198624045,5.25,8.50


In [21]:
pract_json = requests.get(sb_url + '/stock/market/batch?symbols={}&types=chart,quote,company,stats&range=day&exactDate=20200302&chartByDay=true&filter=industry,chart,latestPrice,marketcap&token='.format('ramp,otis,msft') + sb_token).json()
ramp_df = pd.DataFrame(pract_json).T.apply(clean_json_df_two, axis = 1)
ramp_df

,company,stats,quote,chart
RAMP,Prv DSscicn seaegoiertsa,2491358256,38.26,0.00
OTIS,gdlstuBdiruioPnc,21628215096,48.29,0.00
MSFT,SteaoPagdcrwf aek,1370840527132,183.57,173.95


In [272]:
d

,Company,Stats,Quote,Chart
MSFT,etegacw koSdafPra,1381913955506,174.78,176.76
AAPL,eleuen omucntEtiasqiminmopcT,1327580575168,296.45,309.88
AMZN,enntRt relItaie,1148424890568,"2,377.26","2,025.66"
FB,eSanresrevt/ew cnfSeitIrot,600174529638,209.38,205.10
GOOGL,riefn/esoeIwetaStcSnvtr er,910744116323,"1,381.18","1,425.58"
...,...,...,...,...
DDD,hnraiIit ecsdanMyrul,963147829,8.15,9.95
BHE,oclcsto eitCprmnoeEnn,748962019,20.54,28.28
XHR,tentRet muaEasl enTets vrIsts,1072561005,9.20,16.39
FBP,nBlkgReons iaa,1190324294,5.59,8.60


In [29]:
d_index_lower = [x.lower() for x in d.index]
d_index_lower

['msft',
 'aapl',
 'amzn',
 'fb',
 'googl',
 'goog',
 'jnj',
 'brk.b',
 'v',
 'jpm',
 'pg',
 'unh',
 'intc',
 'ma',
 'hd',
 'vz',
 't',
 'pfe',
 'mrk',
 'dis',
 'xom',
 'bac',
 'pep',
 'csco',
 'nvda',
 'ko',
 'nflx',
 'cvx',
 'cmcsa',
 'wmt',
 'adbe',
 'abt',
 'pypl',
 'bmy',
 'crm',
 'mcd',
 'amgn',
 'cost',
 'tmo',
 'mdt',
 'lly',
 'abbv',
 'acn',
 'pm',
 'orcl',
 'wfc',
 'nee',
 'tsla',
 'ibm',
 'unp',
 'txn',
 'nke',
 'c',
 'avgo',
 'gild',
 'dhr',
 'amt',
 'hon',
 'lin',
 'rtx',
 'lmt',
 'sbux',
 'qcom',
 'mmm',
 'fis',
 'low',
 'cvs',
 'chtr',
 'mo',
 'ba',
 'mdlz',
 'ci',
 'spgi',
 'antm',
 'intu',
 'bdx',
 'ups',
 'pld',
 'cat',
 'cme',
 'd',
 'cci',
 'axp',
 'vrtx',
 'bkng',
 'adp',
 'duk',
 'amd',
 'tjx',
 'agn',
 'syk',
 'now',
 'so',
 'blk',
 'gs',
 'cl',
 'isrg',
 'fisv',
 'zts',
 'eqix',
 'ge',
 'tgt',
 'usb',
 'mu',
 'biib',
 'csx',
 'tfc',
 'bsx',
 'noc',
 'gpn',
 'nem',
 'hum',
 'cb',
 'ice',
 'apd',
 'amat',
 'atvi',
 'ecl',
 'pnc',
 'itw',
 'mmc',
 'nsc',
 'kmb',
 '

In [30]:
len(ticker_list)

1500

In [31]:
for x in ticker_list:
    if x not in d_index_lower:
        print(x)
        print(ticker_list.index(x))
## xtsla is a fund. Not sure why others are not working. 

xtsla
162
bfb
401
msfut
617
heia
640
bfa
900
moga
1209
jwa
1240


In [347]:
# Do I fix so its one at a time? Maybe
# Below is code to fix one at a time (retrieve each company one at a time)
pract_json = requests.get(sb_url + '/stock/market/batch?symbols={}&types=chart,quote,company,stats&range=day&exactDate=20200302&chartByDay=true&filter=industry,chart,latestPrice,marketcap&token='.format('xtsla') + sb_token).json()
clean_single_company(pd.DataFrame(pract_json).T)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [330]:
def clean_single_company(df):
    try:
        df['chart'] = df['chart'][0][0]['close']
    except:
        df['chart'] = 0
    df['company'] = df['company'][0]['industry']
    df['quote'] = df['quote'][0]['latestPrice']
    df['stats'] = df['stats'][0]['marketcap']
    return df
    

In [343]:
ticker_list = list(itot_top_1500['Ticker'].apply(lambda x: x.lower()))
d_two = pd.DataFrame()
for x in ticker_list:
    try:
        json = requests.get(sb_url + '/stock/market/batch?symbols={}&types=chart,quote,company,stats&range=day&exactDate=20200302&chartByDay=true&filter=industry,chart,latestPrice,marketcap&token='.format(x) + sb_token).json()
        d_two = d_two.append(clean_single_company(pd.DataFrame(json).T))
    except:
        print(x)

# This worked!! Can it be replicated with REAL data?

xtsla
bfb
msfut
heia
bfa
moga
jwa


In [35]:
requests.get(sb_url + '/stock/xtsla/quote?token=' + sb_token).text

'Unknown symbol'

In [36]:
d = pd.DataFrame()
#for i in range(0,len(ticker_list),50):
#    url = ','.join(ticker_list[i: i + 50])
#    json = requests.get(base_url + '/stock/market/batch?symbols={}&types=chart,quote,company,stats&range=day&exactDate=20200302&chartByDay=true&filter=industry,chart,latestPrice,marketcap&token='.format(url) + token).json()
#    df = iex_dataframe_two(json)
#    d = d.append(df)

In [37]:
d

,Company,Stats,Quote,Chart
MSFT,Packaged Software,1356222409600,180.76,172.79
AAPL,Telecommunications Equipment,1270655114400,297.56,298.81
AMZN,Internet Retail,1155160228240,"2,317.80","1,953.95"
FB,Internet Software/Services,584898633000,207.07,196.44
GOOGL,Internet Software/Services,903037324644,"1,349.02","1,386.32"
...,...,...,...,...
DDD,Industrial Machinery,975898070,8.33,9.53
BHE,Electronic Components,723221760,19.43,27.56
XHR,Real Estate Investment Trusts,1041067080,8.50,15.61
FBP,Regional Banks,1175887790,5.21,8.30


In [38]:
d.to_csv('iexcloud_data.csv', index = True)